In [1]:
import torch
import gym
from gym import envs
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
# from torch.autograd import Variable
from torch.distributions import Categorical
# from torchvision import transforms
# from torch.utils.tensorboard import SummaryWriter
import concurrent.futures


In [66]:
lenobs = 100800
class ActorCritic(nn.Module):
    def __init__(self, ran):
        super(ActorCritic, self).__init__()
        torch.random.manual_seed(ran)
        self.l1 = nn.Linear(lenobs,25)
        self.l2 = nn.Linear(25,50)
        self.actor_lin1 = nn.Linear(50,6)
        self.l3 = nn.Linear(50,25)
        self.critic_lin1 = nn.Linear(25,1)

    def forward(self,x):
        x = F.normalize(x,dim=0)
        y = F.relu(self.l1(x))
        y = F.normalize(y,dim=0)
        y = F.relu(self.l2(y))
        y = F.normalize(y,dim=0)
#         actor = F.softmax(self.actor_lin1(y),dim=0)
        actor = F.log_softmax(self.actor_lin1(y),dim=0)
        c = F.relu(self.l3(y.detach()))
        critic = F.hardtanh(self.critic_lin1(c))
        return actor, critic

In [235]:
def doTrain(model, ran):  
    
    env = gym.make('Pong-v0')
    mi = model(ran)
#     optimizer = optim.Adam(lr=1e-4,params=mi.parameters())
    
    mi.train()
    values = []
    rewards = []
    logprobs = []
    observation = env.reset()
    done = False
    N = 0
    while done == False and N<10:
        N+=1
        pobservation = torch.from_numpy(observation)
        flattened_pobservation = pobservation.view(-1).float()
        policy, value = mi(flattened_pobservation)
        values.append(value.item())
        sampler = Categorical(policy)
        action = sampler.sample()
        logprobs.append(policy[action.item()].item())
        observation, reward, done, log = env.step(action.item())
        if done:
            rewards.append(1.0)
        else:
            rewards.append(reward)
        
    torch_values = torch.tensor(values, requires_grad = True).view(-1).flip(0)
    torch_rewards = torch.tensor(rewards, requires_grad = True).flip(0)
    torch_logprobs = torch.tensor(logprobs, requires_grad = True).flip(0)
    
    returns = []
    gamma = 0.90
    clc = 0.1
    ret = torch.tensor([0])
    for r in torch_rewards:
        ret = r + gamma*ret
        returns.append(ret)
    returns = torch.tensor(returns, requires_grad = True)
    returns = F.normalize(returns,dim=0)
    actor_loss = -1*torch_logprobs * (returns - torch_values.detach())
    critic_loss = torch.pow(torch_values - returns,2)
    loss = actor_loss.sum() + clc*critic_loss.sum()
#     optimizer.zero_grad()
    loss.backward()
    gradients = []
    
    for i in mi.parameters():
        gradients.append(i.grad)
    
#     optimizer.step()
    return gradients


In [236]:
updatedParams = []
results = []


with concurrent.futures.ProcessPoolExecutor() as executor:
    for i in range(5):
        results.append(executor.submit(doTrain, ActorCritic, int((torch.randn(1)**2)*200)))
    

    for f in concurrent.futures.as_completed(results):
        updatedParams.append(f.result())


In [237]:
updatedParams

[[None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None, None]]

In [230]:
class mymodel(nn.Module):
    def __init__(self):
        super(mymodel, self).__init__()
        self.w1 = nn.Linear(3,2)
        
    def forward(self, X):
        out = self.w1(X)
        return out
    
    
# a1 = mymodel()
# op = optim.Adam(lr=1e-3, params = a1.parameters())

    
X = torch.randn(3)
out = a1(X)
op.zero_grad()
loss = out.sum()
loss.backward()
print(loss)
op.step()

tensor(-0.7638, grad_fn=<SumBackward0>)


In [231]:
for i in a1.parameters():
    print(i.grad)

tensor([[-0.6394, -0.9047,  0.2594],
        [-0.6394, -0.9047,  0.2594]])
tensor([1., 1.])
